# 🎙️ Kokoro TTS for Kaggle v2

**v2 Fix:** Better CORS handling for production sites

**Steps:**
1. Enable GPU: Settings → Accelerator → GPU T4 x2
2. Enable Internet: Settings → Internet → ON
3. ⚠️ Verify phone if needed for GPU access
4. Run cells one by one (Shift+Enter)
5. Copy ngrok URL

In [ ]:
!pip install -q kokoro>=0.9.4 flask flask-cors pyngrok soundfile numpy torch

In [ ]:
from kokoro import KPipeline
import torch

print("🔄 Loading Kokoro model...")
print(f"   GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

pipeline = KPipeline(lang_code='a')

print("✅ Model loaded!")

In [ ]:
# Enter your ngrok token here!
NGROK_TOKEN = ""  # <-- PASTE YOUR TOKEN HERE

from pyngrok import ngrok
if NGROK_TOKEN:
    ngrok.set_auth_token(NGROK_TOKEN)
    print("✅ ngrok token set!")
else:
    print("⚠️ Enter your token above! Get one at: https://dashboard.ngrok.com/get-started/your-authtoken")

In [ ]:
from flask import Flask, request, send_file, jsonify, Response
from flask_cors import CORS
import soundfile as sf
import numpy as np
import io, json, base64

app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "*"}})

# Explicit CORS headers for all responses (fixes ngrok CORS issues)
@app.after_request
def after_request(response):
    response.headers.add('Access-Control-Allow-Origin', '*')
    response.headers.add('Access-Control-Allow-Headers', 'Content-Type, ngrok-skip-browser-warning')
    response.headers.add('Access-Control-Allow-Methods', 'GET, POST, OPTIONS')
    return response

# Handle preflight OPTIONS requests explicitly
@app.route('/health', methods=['GET', 'OPTIONS'])
def health():
    if request.method == 'OPTIONS':
        return '', 200
    return jsonify({"status": "ok", "model": "kokoro", "version": "v2", "gpu": torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU"})

@app.route('/api/voices', methods=['GET', 'OPTIONS'])
def get_voices():
    if request.method == 'OPTIONS':
        return '', 200
    VOICES = {
        "af_bella": "Bella", "af_nicole": "Nicole", "af_sarah": "Sarah",
        "af_sky": "Sky", "am_adam": "Adam", "am_michael": "Michael",
        "bf_emma": "Emma", "bm_george": "George"
    }
    return jsonify(VOICES)

@app.route('/api/tts', methods=['POST', 'OPTIONS'])
def generate_tts():
    if request.method == 'OPTIONS':
        return '', 200
    
    data = request.json
    text = data.get('text', 'Hello')
    voice = data.get('voice', 'af_bella')
    speed = float(data.get('speed', 1.0))
    stream = data.get('stream', False)
    
    if not stream:
        try:
            print(f"🎙️ Gen: voice={voice}, speed={speed}, chars={len(text)}")
            generator = pipeline(text, voice=voice, speed=speed)
            audio_chunks = [audio for _, _, audio in generator]
            final = torch.cat(audio_chunks).numpy()
            
            buffer = io.BytesIO()
            sf.write(buffer, final, 24000, format='WAV')
            buffer.seek(0)
            print(f"✅ Generated {len(final)/24000:.2f}s")
            return send_file(buffer, mimetype='audio/wav')
        except Exception as e:
            return jsonify({"error": str(e)}), 500
    
    def generate_with_progress():
        try:
            print(f"🎙️ [Stream] voice={voice}, speed={speed}")
            estimated = max(1, len(text) // 100)
            yield f"data: {json.dumps({'type': 'progress', 'current': 0, 'total': estimated, 'percent': 0, 'status': 'Starting...'})}\n\n"
            
            generator = pipeline(text, voice=voice, speed=speed)
            audio_chunks = []
            count = 0
            
            for _, _, audio in generator:
                audio_chunks.append(audio)
                count += 1
                pct = min(95, int((count / max(estimated, count)) * 100))
                yield f"data: {json.dumps({'type': 'progress', 'current': count, 'total': max(estimated, count), 'percent': pct, 'status': f'Chunk {count}...'})}\n\n"
            
            final = torch.cat(audio_chunks).numpy()
            buffer = io.BytesIO()
            sf.write(buffer, final, 24000, format='WAV')
            buffer.seek(0)
            audio_b64 = base64.b64encode(buffer.read()).decode('utf-8')
            
            yield f"data: {json.dumps({'type': 'complete', 'audio': audio_b64, 'duration': len(final)/24000})}\n\n"
        except Exception as e:
            yield f"data: {json.dumps({'type': 'error', 'message': str(e)})}\n\n"
    
    return Response(generate_with_progress(), mimetype='text/event-stream', headers={'Cache-Control': 'no-cache', 'X-Accel-Buffering': 'no'})

public_url = ngrok.connect(5000)
print("\n" + "="*50)
print(f"🚀 KOKORO TTS URL: {public_url}")
print("   Version: v2 (fixed CORS)")
print("="*50 + "\n")

app.run(port=5000)